In [11]:
# Importando as bibliotecas necessárias
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from imblearn.over_sampling import SMOTE
from sklearn.metrics import f1_score
import matplotlib.pyplot as plt 

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.preprocessing import StandardScaler

from tensorflow.keras.layers import Dense, Dropout 
from tensorflow.keras.callbacks import EarlyStopping 
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import RandomizedSearchCV


In [21]:
df_final_total = pd.read_parquet(r"../app/data/silver/df_ML_tunado.parquet")

In [22]:

print("\nDataFrame final para o modelo salvo como 'df_ML_tunado.parquet'")
print(f"O DataFrame contém {df_final_total.shape[0]} linhas e {df_final_total.shape[1]} colunas.")



DataFrame final para o modelo salvo como 'df_ML_tunado.parquet'
O DataFrame contém 12211 linhas e 74 colunas.


In [13]:
# Separando am base de treino e teste

# A variável 'y' é a coluna que queremos prever (o nosso alvo).
y = df_final_total['status_geral_codificado']

colunas_para_remover = ['status_geral_codificado', 'id_vaga', 'dict_prospect_codigo']
X = df_final_total.drop(columns=colunas_para_remover)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Utilizar o smote para equilibrar as classes

smote = SMOTE(random_state=42)
X_train, y_train = smote.fit_resample(X_train, y_train)


In [14]:
# rede neural (Mult Layer Perceptron)

# --- PASSO 1: Escalonamento dos Dados ---
# Criamos o objeto scaler
scaler = StandardScaler()

# Ajuste o scaler APENAS nos dados de treino
X_train = scaler.fit_transform(X_train)

# Use o scaler já ajustado para TRANSFORMAR os dados de teste
X_test = scaler.transform(X_test)

# --- PASSO 2: Construção da Arquitetura do Modelo ---
# O número de features de entrada é o número de colunas em X
n_features = X_train.shape[1]

# Usamos o modelo Sequential, que nos permite empilhar camadas uma a uma
modelo_nn = Sequential()

# Camada de Entrada e Primeira Camada Oculta
modelo_nn.add(Dense(64, activation='relu', input_shape=(n_features,)))
modelo_nn.add(Dropout(0.4)), # Desliga 50% dos neurônios para combater overfitting

# Segunda Camada Oculta
modelo_nn.add(Dense(32, activation='relu'))
modelo_nn.add(Dropout(0.2)), # Desliga 30% dos neurônios

modelo_nn.add(Dense(16, activation='relu')), # <-- Nova camada

# Camada de Saída
modelo_nn.add(Dense(1, activation='sigmoid'))

# --- PASSO 3: Compilação do Modelo ---
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
modelo_nn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Mostra um resumo da arquitetura criada
print("--- Arquitetura da Rede Neural ---")
modelo_nn.summary()



--- Arquitetura da Rede Neural ---


C:\Users\leand\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\core\dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_4 (Dense)                 │ (None, 64)             │         4,608 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 7,233 (28.25 KB)

 Trainable params: 7,233 (28.25 KB)

 Non-trainable params: 0 (0.00 B)

In [15]:
# Configura o callback de Parada Antecipada
callback_early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)


# --- PASSO 4: Treinamento do Modelo ---
print("\n--- Iniciando o Treinamento da Rede Neural ---")
historico = modelo_nn.fit(
    X_train, 
    y_train, 
    epochs=100, 
    batch_size=32, 
    validation_data=(X_test, y_test),
    verbose=1 # verbose=1 mostra uma barra de progresso
)




--- Iniciando o Treinamento da Rede Neural ---
Epoch 1/100
451/451 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.5674 - loss: 0.7331 - val_accuracy: 0.7122 - val_loss: 0.5894
Epoch 2/100
451/451 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6926 - loss: 0.5850 - val_accuracy: 0.7233 - val_loss: 0.5871
Epoch 3/100
451/451 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7138 - loss: 0.5657 - val_accuracy: 0.7352 - val_loss: 0.5797
Epoch 4/100
451/451 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7133 - loss: 0.5524 - val_accuracy: 0.7438 - val_loss: 0.5575
Epoch 5/100
451/451 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7189 - loss: 0.5391 - val_accuracy: 0.7315 - val_loss: 0.5585
Epoch 6/100
451/451 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7224 - loss: 0.5335 - val_accuracy: 0.7393 - val_loss: 0.5581
Epoch 7/100
451/451 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7336 - loss: 0.5229 - val_accuracy: 0.7397 - val_loss: 0.5601
Epoch 8/100
451/451 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/s

In [16]:
# --- PASSO 5: Avaliação Final do Modelo ---
print("\n--- Avaliação Final do Modelo nos Dados de Teste ---")

# .predict() retorna as probabilidades de pertencer à classe 1
probabilidades = modelo_nn.predict(X_test)
# Convertemos as probabilidades em classes (0 ou 1) usando um limiar de 0.5
y_pred_nn = (probabilidades > 0.5).astype(int)

print("\nRelatório de Classificação da Rede Neural:")
print(classification_report(y_test, y_pred_nn))


--- Avaliação Final do Modelo nos Dados de Teste ---
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step  

Relatório de Classificação da Rede Neural:
              precision    recall  f1-score   support

           0       0.53      0.48      0.50       646
           1       0.82      0.85      0.83      1797

    accuracy                           0.75      2443
   macro avg       0.67      0.66      0.67      2443
weighted avg       0.74      0.75      0.74      2443



In [17]:
# 1. Obter as probabilidades usando o método .predict()

print("Gerando probabilidades com a Rede Neural...")
probabilidades_reprovado = modelo_nn.predict(X_test)

prob_ser_aprovado = 1 - probabilidades_reprovado.flatten()

# 3. Criar um novo DataFrame para o ranking
df_ranking = X_test.copy()

df_ranking = pd.DataFrame(X_test, columns=X.columns)

# 4. Adicionar as informações importantes para análise
df_ranking['probabilidade_match'] = prob_ser_aprovado
df_ranking['status_real'] = y_test # Adiciona o resultado real para comparação

# Para facilitar a identificação, vamos buscar os IDs originais usando o índice do DataFrame
df_ranking = df_ranking.join(df_final_total[['id_vaga', 'dict_prospect_codigo']])

# 5. Ordenar o ranking pela probabilidade, do maior para o menor
df_ranking_final = df_ranking.sort_values(by='probabilidade_match', ascending=False)
df_ranking_final = df_ranking_final[df_ranking_final['id_vaga'].notna()]

# 6. Exibir o resultado
print("\n--- Ranking de Candidatos (Gerado pela Rede Neural) ---")
colunas_para_exibir = ['id_vaga', 'dict_prospect_codigo', 'probabilidade_match', 'status_real']
df_ranking_final[colunas_para_exibir].head(20)

Gerando probabilidades com a Rede Neural...
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 923us/step

--- Ranking de Candidatos (Gerado pela Rede Neural) ---


,id_vaga,dict_prospect_codigo,probabilidade_match,status_real
1365,13194,48403,0.999997,NaN
272,5753,18889,0.999994,NaN
1082,3829,23116,0.999985,1.0
433,6571,29583,0.999896,NaN
439,6606,29592,0.998802,0.0
1326,2881,21073,0.998183,NaN
216,5387,26837,0.997618,1.0
2315,2369,19268,0.996547,NaN
918,10100,7809,0.996111,NaN
1305,2794,20773,0.992494,NaN
